In [2]:
!rm -rf ViT

In [7]:
!git clone https://github.com/christianodenwald/ViT.git
%pip install einops

fatal: destination path 'ViT' already exists and is not an empty directory.


In [4]:
import sys
sys.path.append("ViT")

In [5]:
from vit import ViT
from vit import test

In [6]:
test("hello world")

this is a test: hello world


The code starts here:

In [1]:
# Training settings
batch_size = 64
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

In [ ]:
# data loading
train_data = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=normalize_images)
test_data = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=normalize_images)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
